In [1]:
pip install numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output

# --------------------------
# --- Grid & Parameters ---
# --------------------------
hbar = 1.0
m = 1.0
L = 10.0
Nx = 128
Ny = 128
dx = L/Nx
dy = L/Ny
dt = 0.001
Nt = 200

x = np.linspace(0, L, Nx)
y = np.linspace(0, L, Ny)
X, Y = np.meshgrid(x, y)

# --------------------------
# --- Potentials ---
# --------------------------
def harmonic_potential(X, Y, k=50):
    return 0.5 * k * ((X-L/2)**2 + (Y-L/2)**2)

def double_well_potential(X, Y, V0=200):
    return V0 * (((X-L/2)**2 - 1)**2 + ((Y-L/2)**2 - 1)**2)

# --------------------------
# --- Wave Packet ---
# --------------------------
def gaussian_wave_packet(X, Y, x0, y0, kx0, ky0, sigma):
    return np.exp(-((X-x0)**2 + (Y-y0)**2)/(2*sigma**2)) * np.exp(1j*(kx0*X + ky0*Y))

# --------------------------
# --- Probability Current ---
# --------------------------
def probability_current(psi, dx, dy, hbar=1.0, m=1.0):
    dpsi_dx = np.zeros_like(psi, dtype=complex)
    dpsi_dy = np.zeros_like(psi, dtype=complex)
    dpsi_dx[1:-1, :] = (psi[2:, :] - psi[:-2, :])/(2*dx)
    dpsi_dy[:, 1:-1] = (psi[:, 2:] - psi[:, :-2])/(2*dy)
    Jx = (hbar/m) * np.imag(np.conj(psi)*dpsi_dx)
    Jy = (hbar/m) * np.imag(np.conj(psi)*dpsi_dy)
    return Jx, Jy

# --------------------------
# --- Split-Step FFT Setup ---
# --------------------------
kx = 2*np.pi*np.fft.fftfreq(Nx, d=dx)
ky = 2*np.pi*np.fft.fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky)

# --------------------------
# --- TDSE Simulation Function ---
# --------------------------
def run_tdse(potential_type, k_strength, sigma, kx0, ky0):
    clear_output(wait=True)  # Clear previous output
    
    # --- Potential ---
    if potential_type=='Harmonic':
        V = harmonic_potential(X, Y, k=k_strength)
    else:
        V = double_well_potential(X, Y, V0=k_strength)
    
    expV = np.exp(-1j*V*dt/(2*hbar))
    expK = np.exp(-1j*hbar*(KX**2 + KY**2)*dt/(2*m))
    
    # --- Initial Wave Function ---
    psi = (gaussian_wave_packet(X, Y, L/3, L/3, kx0, ky0, sigma) +
           gaussian_wave_packet(X, Y, 2*L/3, 2*L/3, -kx0, -ky0, sigma))
    psi /= np.sqrt(np.sum(np.abs(psi)**2)*dx*dy)
    
    psi_anim = psi.copy()
    
    # --- 2D Heatmap Animation ---
    fig, ax = plt.subplots(figsize=(6,5))
    heatmap = ax.imshow(np.abs(psi_anim)**2, extent=[0,L,0,L], origin='lower', cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('2D TDSE |ψ|²')
    plt.colorbar(heatmap)
    
    def evolve(frame):
        nonlocal psi_anim
        psi_anim = expV*psi_anim
        psi_k = np.fft.fft2(psi_anim)
        psi_k = expK*psi_k
        psi_anim = np.fft.ifft2(psi_k)
        psi_anim = expV*psi_anim
        heatmap.set_data(np.abs(psi_anim)**2)
        return heatmap,
    
    anim = FuncAnimation(fig, evolve, frames=Nt, interval=50, blit=False)
    plt.show()
    
    # --- 3D Surface ---
    fig2 = plt.figure(figsize=(7,5))
    ax2 = fig2.add_subplot(111, projection='3d')
    surf = ax2.plot_surface(X, Y, np.abs(psi_anim)**2, cmap='viridis')
    ax2.set_xlabel('x'); ax2.set_ylabel('y'); ax2.set_zlabel('|ψ|²')
    ax2.set_title('Final 2D TDSE Surface')
    plt.show()
    
    # --- Probability Current ---
    Jx, Jy = probability_current(psi_anim, dx, dy)
    skip = (slice(None,None,8), slice(None,None,8))
    fig3, ax3 = plt.subplots(figsize=(6,5))
    ax3.imshow(np.abs(psi_anim)**2, extent=[0,L,0,L], origin='lower', cmap='viridis')
    ax3.quiver(X[skip], Y[skip], Jx[skip], Jy[skip], color='white', scale=0.1)
    ax3.set_xlabel('x'); ax3.set_ylabel('y'); ax3.set_title('Probability Current')
    plt.show()

# --------------------------
# --- Widgets ---
# --------------------------
potential_widget = widgets.Dropdown(options=['Harmonic','Double Well'], description='Potential')
k_strength_widget = widgets.FloatSlider(min=10, max=200, step=5, value=50, description='Strength')
sigma_widget = widgets.FloatSlider(min=0.1, max=1.0, step=0.05, value=0.3, description='σ')
kx0_widget = widgets.FloatSlider(min=0, max=50, step=1, value=20, description='kx0')
ky0_widget = widgets.FloatSlider(min=0, max=50, step=1, value=0, description='ky0')
run_button = widgets.Button(description='Run TDSE')

ui = widgets.VBox([potential_widget, k_strength_widget, sigma_widget, kx0_widget, ky0_widget, run_button])
display(ui)

def on_run_clicked(b):
    run_tdse(
        potential_widget.value,
        k_strength_widget.value,
        sigma_widget.value,
        kx0_widget.value,
        ky0_widget.value
    )

run_button.on_click(on_run_clicked)
